In [1]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Reading from CSV Files")
    .master("local[*]")
    .getOrCreate()
)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 13:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read a csv file into dataframe

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load("/Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference notes/datasets/emp.csv")

# to identify datatype of each of the column we use inferschema in spark

In [3]:
df.printSchema()

root
 |-- version https://git-lfs.github.com/spec/v1: string (nullable = true)



In [4]:
df.show()

+------------------------------------------+
|version https://git-lfs.github.com/spec/v1|
+------------------------------------------+
|                      oid sha256:2a7c3d...|
|                                  size 968|
+------------------------------------------+



In [7]:
# Reading with Schema
_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date"

df_schema = spark.read.format("csv").option("header",True).schema(_schema).load("/Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference notes/datasets/emp.csv")

# In Production always use _schema so that spark doesn't have to collect metadata of the file each time we run the query and also it will help spark to optimize the code

In [8]:
df_schema.show()

+-----------+-------------+----+----+------+------+---------+
|employee_id|department_id|name| age|gender|salary|hire_date|
+-----------+-------------+----+----+------+------+---------+
|       NULL|         NULL|NULL|NULL|  NULL|  NULL|     NULL|
|       NULL|         NULL|NULL|NULL|  NULL|  NULL|     NULL|
+-----------+-------------+----+----+------+------+---------+



24/07/24 13:31:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp.csv


In [9]:
# Most important mode in CSV reading is mode.
# It is very useful to handle bad records. Schema is necessary.
# There are three modes
# 1. PERMISSIVE 
# 2. DROPMALFORMED
# 3. FAILFAST

In [16]:
# Handle BAD records - PERMISSIVE (Default mode)

_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date, bad_record string"

df_p = spark.read.format("csv").schema(_schema).option("columnNameOfCorruptRecord", "bad_record").option("header", True).load("/Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference notes/datasets/emp_new.csv")


# Interview Question?

# In this scenario how many jobs Spark created in this query?

# Answer: Only one because spark already has data Schema into account and it also knows the metadata of the dataset through the schema

In [17]:
df_p.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- bad_record: string (nullable = true)



In [19]:
df_p.show()

+-----------+-------------+----+----+------+------+---------+--------------------+
|employee_id|department_id|name| age|gender|salary|hire_date|          bad_record|
+-----------+-------------+----+----+------+------+---------+--------------------+
|       NULL|         NULL|NULL|NULL|  NULL|  NULL|     NULL|oid sha256:c0bdc7...|
|       NULL|         NULL|NULL|NULL|  NULL|  NULL|     NULL|            size 963|
+-----------+-------------+----+----+------+------+---------+--------------------+



24/07/24 13:48:08 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp_new.csv


In [23]:
# Handle BAD records - DROPMALFORMED, # Drops the Bad Record
_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date"

df_m = spark.read.format("csv").option("header", True).option("mode", "DROPMALFORMED").schema(_schema).load("/Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference notes/datasets/emp_new.csv")


In [24]:
df_m.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)



In [25]:
df_m.show()

+-----------+-------------+----+---+------+------+---------+
|employee_id|department_id|name|age|gender|salary|hire_date|
+-----------+-------------+----+---+------+------+---------+
+-----------+-------------+----+---+------+------+---------+



24/07/24 13:51:04 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp_new.csv


In [27]:
# Handle BAD records - FAILFAST # in this mode When bad records get identified the job fails.

_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date"

df_m = spark.read.format("csv").option("header", True).option("mode", "FAILFAST").schema(_schema).load("/Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference notes/datasets/emp_new.csv")



In [28]:
df_m.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)



In [29]:
df_m.show()

24/07/24 13:52:21 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 1, schema size: 7
CSV file: file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp_new.csv
24/07/24 13:52:21 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 11)
org.apache.spark.SparkException: Encountered error while reading file file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp_new.csv. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:863)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStag

Py4JJavaError: An error occurred while calling o160.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11) (vinayaks-mbp.station executor driver): org.apache.spark.SparkException: Encountered error while reading file file:///Users/vinayakpawar/Desktop/Work/Full_Stack/Python/PySpark/Reference%20notes/datasets/emp_new.csv. Details:
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:863)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:293)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIte

In [ ]:
# BONUS TIP
# Multiple options

_options = {
    "header" : "true",
    "inferSchema" : "true",
    "mode" : "PERMISSIVE"
}

df = (spark.read.format("csv").options(**_options).load("data/input/emp.csv"))


In [ ]:
df.show()

In [ ]:
spark.stop()